In [11]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
import pmdarima as pm
from pmdarima.model_selection import train_test_split
from statsforecast import StatsForecast
from statsforecast.models import (AutoARIMA, DynamicOptimizedTheta as DOT, WindowAverage)
from multiprocessing import cpu_count
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError
from datasetsforecast.losses import rmse

Load data

In [76]:
sales = pd.read_csv('../data/sales_products_ts.csv')
products_info = pd.read_csv("../data/products_statistics.csv")


Error functions

In [12]:
#def rmse(y_true, y_pred):
#    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

# metric for interminent demand series
mase = MeanAbsoluteScaledError()

Preprocessing function

In [77]:
def preprocessing(df):
    sorted = df.groupby('id').sum('quantity').sort_values('quantity', ascending=False)
    id_list = sorted.index.unique()
    df = df.rename(columns={'date':'ds', 'quantity':'y'})
    df = df.set_index('id')
    df = df.rename_axis('unique_id')
    df.ds = pd.to_datetime(df.ds)
    df = df.sort_values(by=['unique_id', 'ds'])
    df = df[['ds','y']]
    #df = df.drop_duplicates()
    return df, id_list


In [78]:
#sales = sales[sales['quantity']!=0]
sales, id_list = preprocessing(sales)

serie original sin preprocessing

In [75]:
df = sales[sales['id']=='17410306']
#df[df['quantity']==0]
df

C:\Users\fcamargo\AppData\Local\Temp\ipykernel_1052\3282503341.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Unnamed: 0,id,date,quantity,sale_week


serie con preprocessing

In [57]:
df = sales.loc[['17410306']]
df.to_csv('id.csv')

One serie

Arima: weekly, seasonality mensual

In [ ]:
df = pd.read_csv('https://datasets-nixtla.s3.amazonaws.com/air-passengers.csv')
# define models
models = [AutoARIMA(season_length = 12), DOT(season_length = 12)]
# initialize SF object
sf = StatsForecast(
    df = df,
    models = models,
    freq = 'M'
)
crossvalidation_df = sf.cross_validation(
    df = df,
    h = 6, # test size
    step_size=7, # gap including test size
    n_windows=4 # number of folds
)
crossvalidation_df
crossvalidation_df.rename(columns = {'y' : 'actual'}, inplace = True) # rename actual values 

cutoff = crossvalidation_df['cutoff'].unique()

for k in range(len(cutoff)): 
    cv = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff[k]]
    StatsForecast.plot(df, cv.loc[:, cv.columns != 'cutoff'])

rmse = rmse(crossvalidation_df['actual'], crossvalidation_df['AutoARIMA'])
rmse

In [79]:
df = sales.loc[['17410306']]
# define models
models = [AutoARIMA(season_length = 4), DOT(season_length = 4), WindowAverage(window_size=4)]
# initialize SF object
sf = StatsForecast(
    df = df,
    models = models,
    freq = 'W'
)
crossvalidation_df = sf.cross_validation(
    df = df,
    h = 12, # test size
    step_size=16, # gap including test size
    n_windows=4 # number of folds
)
crossvalidation_df

cutoff = crossvalidation_df['cutoff'].unique()

for k in range(len(cutoff)): 
    cv = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff[k]]
    StatsForecast.plot(df, cv.loc[:, cv.columns != 'cutoff'])

rmsee = rmse(crossvalidation_df['y'], crossvalidation_df['AutoARIMA'])
rmse

<__array_function__ internals>:200: RuntimeWarning:

invalid value encountered in cast



<function datasetsforecast.losses.rmse(y: numpy.ndarray, y_hat: numpy.ndarray, weights: Optional[numpy.ndarray] = None, axis: Optional[int] = None) -> Union[float, numpy.ndarray]>

sesonality anual para arima

In [ ]:
df = sales.loc[['17410306']]
# define models
models = [AutoARIMA(season_length = 52), DOT(season_length = 4), WindowAverage(window_size=4)]
# initialize SF object
sf = StatsForecast(
    df = df,
    models = models,
    freq = 'W'
)
crossvalidation_df = sf.cross_validation(
    df = df,
    h = 12, # test size
    step_size=16, # gap including test size
    n_windows=4 # number of folds
)
crossvalidation_df

cutoff = crossvalidation_df['cutoff'].unique()

for k in range(len(cutoff)): 
    cv = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff[k]]
    StatsForecast.plot(df, cv.loc[:, cv.columns != 'cutoff'])

rmsee = rmse(crossvalidation_df['y'], crossvalidation_df['AutoARIMA'])
rmse

In [67]:
mape(crossvalidation_df['y'], crossvalidation_df['AutoARIMA'])

1.919154e+16

Multiple time series

In [ ]:
df = pd.read_csv('https://datasets-nixtla.s3.amazonaws.com/air-passengers.csv')
# define models
models = [AutoARIMA(season_length = 12), DOT(season_length = 12)]
# initialize SF object
sf = StatsForecast(
    models = models,
    freq = 'M'
)

sf.fit(df)
forecast_df = sf.predict(h=12, level=[90]) 
df["ds"]=pd.to_datetime(df["ds"])
sf.plot(df, forecast_df, level=[90])

In [28]:
sf = StatsForecast(
    models = [AutoARIMA(season_length = 4)],
    freq = 'W',
    n_jobs = min(len(sales.index.unique()),cpu_count())
)
sf.fit(m)
forecast_df = sf.predict(h=12, level=[90]) 

In [ ]:
m["ds"]=pd.to_datetime(m["ds"])
sf.plot(m, forecast_df, level=[90])

98.29644

Multiple TimeSeries


In [59]:
#m = sales.loc[['17410306', '20410301']]


In [ ]:
fcst = StatsForecast(
    sales,
    models = [AutoARIMA(season_length = 4)],
    freq = 'W',
    n_jobs = min(len(sales.index.unique()),cpu_count())
)

#%time arima_fc = fcst.forecast(12)

Codigos adicionales del Paquete

In [ ]:
StatsForecast.plot(df) # grafica la serie original
?AutoARIMA # muestra hiperparametros del modelo


In [ ]:
train_complete = []
for item in id_list:
  one_product = sales[sales.index==item]
  split_date, train, test = split(one_product)
  print(train)
  #train_complete = pd.concat([train_complete, train])
